In [1]:
import webcam
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def add_fps(img, timer):
  font                   = cv.FONT_HERSHEY_SIMPLEX
  bottomLeftCornerOfText = (10,30)
  fontScale              = 1
  fontColor              = (255,0,0)
  lineType               = 2

  texts = [
    "Total FPS: " + str(timer.get_fps("total")),
    "Read Time %: " + str(timer.get_time_percentage("read")),
    "Conversion Time %: " + str(timer.get_time_percentage("conversion")),
    "Harris Time %: " + str(timer.get_time_percentage("harris_corner")),
    "Dilation Time %: " + str(timer.get_time_percentage("dilation")),
    "Thresholding Time %: " + str(timer.get_time_percentage("thresholding"))
  ]

  count = 0
  for text in texts:
    if count == 0:
      fontScale = 1
    else:
      fontScale = .5

    bottomLeftCornerOfText = (10, 30 + count*30)
    cv.putText(img, text, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
    count += 1
  return img 

In [3]:
import signal
import sys
def signal_handler(signal, frame):
        print('You pressed Ctrl+C!')
        sys.exit(0)

In [4]:
def show_webcam_harris():
    cam = cv.VideoCapture("imgs/isec_vid.MOV")
    cv.startWindowThread()
    
    fps = cam.get(cv.CAP_PROP_FPS)   #not quite! figure out real fps
    print("Frames per second", fps)  # with timers
    
    
    signal.signal(signal.SIGINT, signal_handler)
    is_interrupted = False
    
    mtime = webcam.TimeRecord()
    while True:
        mtime.start("total")
        mtime.start("read")
        ret_val, img = cam.read()  # read frame
        mtime.end("read")

        if img is None:
            pass

        mtime.start("conversion")
        gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY) #convert to grayscale
        gray = np.float32(gray)                   # convert to float
        mtime.end("conversion")

        mtime.start("harris_corner")
        dst = cv.cornerHarris(gray,2,3,0.04)      # Harris
        mtime.end("harris_corner")

        #result is dilated for marking the corners, not important
        mtime.start("dilation")
        dst = cv.dilate(dst,None)
        mtime.end("dilation")

        # Threshold for an optimal value, it may vary depending on the image.
        mtime.start("thresholding")
        img[dst>0.005*dst.max()]=[0,0,255]
        mtime.end("thresholding")
        mtime.end("total")
        mtime.iterate()

        img = add_fps(img, mtime)
        # load image using cv2....and do processing.
        #plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        # as opencv loads in BGR format by default, we want to show it in RGB.
        #plt.show()
        
        display.clear_output(wait=True)
        display.display(plt.gcf())
        
        vc = cv2.VideoCapture(0)

        plt.ion()

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        webcam_preview.set_data(frame)
        plt.draw()

        try:    # Avoids a NotImplementedError caused by `plt.pause`
            plt.pause(0.05)
        except Exception:
            pass
        if is_interrupted:
            vc.release()
            break
        
    print("Destroying window")
    cv.waitKey(33)
    cv.destroyAllWindows()
    cv.waitKey(33)
    cam.release()
    cv.waitKey(33)

In [5]:
%matplotlib notebook 
show_webcam_harris()

Frames per second 29.977081741787625


AttributeError: 'function' object has no attribute 'clear_output'